# FOM - Anwendungsfelder der Business Analytics am Beispiel von Fraud Detection im von Kreditkartentransaktionen
In diesem Jupyter Note soll ein künstliches neuronales Netz (KNN) erstellt werden, mit welchem Kreditkartentransaktionen auf betrügerische Aktivitäten hin überprüft werden. Für die Umsetzung dieses Vorhabens sollen die in dem Notebook <code>ieee-cis-k-means</code> erstellten Daten und Tensorflow verwendet werden.</br></br>
<b>Folgende Schritte sind zu erledigen</b>
<ol>
    <li>Import der nötigen Python Packages</li>
    <li>Import der vorbereiteten Daten aus AWS S3</li>
    <li>Normalisierung der importierten Daten</li>
</ol>
</br>
Im folgenden werden die zu verwenden Python Packages importiert.

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
from sklearn import preprocessing
from tensorflow import keras
import boto3
import io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
print('{}: {}'.format('Tensorflow version', tf.__version__))

Tensorflow version: 1.14.0


Nachdem die nötigen Python Packages importiert sind, können die zuletzt vorbereiteten Daten aus AWS S3 importiert werden.
<br>
## Import der vorbereiteten Daten aus AWS S3
Für den Import von Daten aus AWS S3 muss zunächst der AWS S3 client initialisiert und der entsprechende Bucket Name angegeben werden.

In [3]:
s3_client = boto3.client('s3')
bucket_name = 'anwendungsfelder-big-data-fraud-detection-ieee-cis-data-set'

Anschließend wird eine Iteration über alle in dem AWS S3 Bucket enthaltende Objekte vorgenommen.

In [4]:
obj_list = s3_client.list_objects(Bucket=bucket_name)
for obj in obj_list:
    print(obj)

Name
ResponseMetadata
MaxKeys
Prefix
Marker
EncodingType
IsTruncated
Contents


Da der Inhalt des Buckets benötigt wird, wird eine Iteration über den Inhalt des Buckets durchgeführt. Da aber das aus der Abfrage zurückkommende Objekt nicht bekannt ist, wird zunächst nur ein Element aus der Liste ausgegeben.

In [5]:
content_list = obj_list['Contents']
print(content_list[0])

{u'LastModified': datetime.datetime(2019, 11, 8, 14, 34, 4, tzinfo=tzlocal()), u'ETag': '"d41d8cd98f00b204e9800998ecf8427e"', u'StorageClass': 'STANDARD', u'Key': u'data/', u'Size': 0}


<code>Key</code> scheint den relativen Pfad zu einem Objekt darzustellen. Im folgenden wird eine Iteration über diesen Key vorgenommen.

In [6]:
i = 0
file_to_retrieve = 'clustering_result.csv'
for content_element in content_list:
    if file_to_retrieve in content_element['Key']: 
        print('{}: {}'.format(str(i), content_element['Key']))
    i += 1

1: data/clustering_result.csv


In [7]:
file_list = []
for content_element in content_list:
    if file_to_retrieve in content_element['Key']:
        file_list.append(content_element['Key'])
print('In Liste vorhandene Element: {}'.format(str(len(file_list))))

In Liste vorhandene Element: 1


In [8]:
s3_file = s3_client.get_object(Bucket=bucket_name, Key=file_list[0])
print('Type of object: {}'.format(str(type(s3_file))))

Type of object: <type 'dict'>


In [9]:
print(s3_file)

{u'Body': <botocore.response.StreamingBody object at 0x7fa01152e110>, u'AcceptRanges': 'bytes', u'ContentType': 'binary/octet-stream', 'ResponseMetadata': {'HTTPStatusCode': 200, 'RetryAttempts': 0, 'HostId': 'lM8EwyBWjmwgu7Lq4tHuZf5k0ILooOM/sn8PXxE7bplkPSsSpAnfph2F0l2H9RodR9LNbZPTiS4=', 'RequestId': '357FA0F10EE72C93', 'HTTPHeaders': {'content-length': '19322227', 'x-amz-id-2': 'lM8EwyBWjmwgu7Lq4tHuZf5k0ILooOM/sn8PXxE7bplkPSsSpAnfph2F0l2H9RodR9LNbZPTiS4=', 'accept-ranges': 'bytes', 'server': 'AmazonS3', 'last-modified': 'Sun, 26 Jan 2020 17:08:04 GMT', 'x-amz-request-id': '357FA0F10EE72C93', 'etag': '"79ad4dde2e2c759c52243eecf613ed88-3"', 'date': 'Sun, 02 Feb 2020 11:30:04 GMT', 'content-type': 'binary/octet-stream'}}, u'LastModified': datetime.datetime(2020, 1, 26, 17, 8, 4, tzinfo=tzutc()), u'ContentLength': 19322227, u'ETag': '"79ad4dde2e2c759c52243eecf613ed88-3"', u'Metadata': {}}


In [10]:
result_file = s3_file['Body'].read()
print('Type of object: {}'.format(str(type(result_file))))

Type of object: <type 'str'>


## Normalisierung der importierten Daten
Da die Daten vorhanden sind können diese in einem <code>DataFrame</code> dargestellt werden.

In [11]:
df = pd.read_csv(io.BytesIO(result_file), header=0)
df.head()

,credit_card,transaction_id,amount_spent,belonging_to_cluster
0,1000.0,3230924.0,23.443001,0.0
1,1001.0,3023634.0,183.000000,0.0
2,1001.0,3210739.0,27.000000,0.0
3,1001.0,3151336.0,29.000000,0.0
4,1004.0,3504379.0,226.000000,0.0


In [12]:
df.shape

(590549, 4)

Bevor die Daten normalisiert werden können, werden diese mit dem tag <code>if_Fraud</code> angereichert. Daher müssen noch weitere Daten importiert werden.

In [13]:
i = 0
file_to_retrieve_2 = 'final_cropped_data.csv'
for content_element in content_list:
    if file_to_retrieve_2 in content_element['Key']: 
        print('{}: {}'.format(str(i), content_element['Key']))
    i += 1

4: data/ieee_cis_train_cropped/Definite+cropped+ieeecis+data/2020/01/20/final_cropped_data.csv


In [14]:
file_list_2 = []
for content_element in content_list:
    if file_to_retrieve_2 in content_element['Key']:
        file_list_2.append(content_element['Key'])
print('In Liste vorhandene Element: {}'.format(str(len(file_list))))

In Liste vorhandene Element: 1


In [15]:
s3_file_2 = s3_client.get_object(Bucket=bucket_name, Key=file_list_2[0])
print('Type of object: {}'.format(str(type(s3_file))))

Type of object: <type 'dict'>


In [16]:
result_file_2 = s3_file_2['Body'].read()
print('Type of object: {}'.format(str(type(result_file))))

Type of object: <type 'str'>


In [17]:
df2 = pd.read_csv(io.BytesIO(result_file_2), header=0)
df2.head()

,credit_card_number,transaction_id,transaction_is_fraudulent,amount_spent,transaction_time_delta,credit_card_holder,credit_card_expiration_date,crdeit_card_validation_code_two
0,1000,3230924,0,23.443,5787419,555.0,185.0,224.0
1,1001,3023634,0,183.000,916268,555.0,150.0,226.0
2,1001,3210739,0,27.000,5270458,555.0,150.0,226.0
3,1001,3151336,0,29.000,3504180,555.0,150.0,226.0
4,1004,3504379,0,226.000,13553567,583.0,150.0,226.0


In [18]:
df2.shape

(590540, 8)

Da nun die Grundlage für die als erste importierten vorliegen, können nun die Datentypen angepasst werden.

In [19]:
df.dtypes

credit_card             float64
transaction_id          float64
amount_spent            float64
belonging_to_cluster    float64
dtype: object

In [20]:
df = df.astype({
    'credit_card' : 'int32',
    'transaction_id' : 'int32',
    })

Nun können die Daten nochmals angereichert werden.

In [21]:
df.head()

,credit_card,transaction_id,amount_spent,belonging_to_cluster
0,1000,3230924,23.443001,0.0
1,1001,3023634,183.000000,0.0
2,1001,3210739,27.000000,0.0
3,1001,3151336,29.000000,0.0
4,1004,3504379,226.000000,0.0


In [22]:
df2 = df2.drop(columns=[
    'credit_card_number',
    'amount_spent'
])

In [23]:
result_df = pd.merge(df, df2, on='transaction_id')
result_df = result_df.drop_duplicates()
result_df.head()

,credit_card,transaction_id,amount_spent,belonging_to_cluster,transaction_is_fraudulent,transaction_time_delta,credit_card_holder,credit_card_expiration_date,crdeit_card_validation_code_two
0,1000,3230924,23.443001,0.0,0,5787419,555.0,185.0,224.0
2,1001,3023634,183.000000,0.0,0,916268,555.0,150.0,226.0
4,1001,3210739,27.000000,0.0,0,5270458,555.0,150.0,226.0
6,1001,3151336,29.000000,0.0,0,3504180,555.0,150.0,226.0
8,1004,3504379,226.000000,0.0,0,13553567,583.0,150.0,226.0


In [24]:
result_df.shape

(590540, 9)

Nun werden die Daten selektiert, die für das Training relevant sind. Dafür wird das Kaufverhalten aller Kreditkarten in sofern geprüft, als das die prozentuale Verteilung der Cluster-Zugehörigkeit untersucht wird. Sollte eine < 3%ige Abweichung des Kaufverhaltens festgestellt werden, so gelten diese als <i>verdächtig</i>. Diese verdächtigen Transaktionen werden dann in einem separaten <code>DataFrame</code> gesammelt und als Input für das Training des KNN verwendet. Entsprechend wird im folgenden ein Algorithmus entwickelt, der diese Transaktionen ermittelt.

In [25]:
credit_cards = result_df.credit_card.unique()
print('Element in credit_cards: {}'.format(str(len(credit_cards))))

Element in credit_cards: 13553


In [26]:
credit_card_df_list = []
for c_card in credit_cards:
    tmp_df = result_df[result_df['credit_card'] == c_card]
    credit_card_df_list.append(tmp_df)
print('Element in credit_card_df_list: {}'.format(str(len(credit_card_df_list))))

Element in credit_card_df_list: 13553


In [27]:
def calculate_precentage(total_count, proportionate_count):
    return (100 / total_count) * proportionate_count

In [28]:
cluster_column = 'belonging_to_cluster'
cluster_0 = 0.0
cluster_1 = 1.0
cluster_2 = 2.0

In [29]:
i = 0
result_data = []
for c_card_df in range(0, len(credit_card_df_list)):
    total = len(credit_card_df_list[c_card_df].index)
    t_belonging_to_cluster_0 = len(credit_card_df_list[c_card_df].loc[credit_card_df_list[c_card_df][cluster_column] == cluster_0])
    t_belonging_to_cluster_1 = len(credit_card_df_list[c_card_df].loc[credit_card_df_list[c_card_df][cluster_column] == cluster_1])
    t_belonging_to_cluster_2 = len(credit_card_df_list[c_card_df].loc[credit_card_df_list[c_card_df][cluster_column] == cluster_2])
    if calculate_precentage(total, t_belonging_to_cluster_0) < 15 and calculate_precentage(total, t_belonging_to_cluster_0) != 0.0:
        result_data.append(credit_card_df_list[c_card_df][credit_card_df_list[c_card_df][cluster_column] == cluster_0])
        i+=1
    elif calculate_precentage(total, t_belonging_to_cluster_0) < 15 and calculate_precentage(total, t_belonging_to_cluster_1) != 0.0:
        result_data.append(credit_card_df_list[c_card_df][credit_card_df_list[c_card_df][cluster_column] == cluster_1])
        i+=1
    elif calculate_precentage(total, t_belonging_to_cluster_0) < 15 and calculate_precentage(total, t_belonging_to_cluster_2) != 0.0:
        result_data.append(credit_card_df_list[c_card_df][credit_card_df_list[c_card_df][cluster_column] == cluster_2])
        i+=1
print('Number of credit cards: {}'.format(str(i)))

Number of credit cards: 723


In [30]:
general_count = 0
fraud = 0
isNotFraud = 0
for element in result_data:
    fraud_list = element['transaction_is_fraudulent'].values.tolist()
    general_count += len(fraud_list)
    for f in fraud_list:
        if f == 0.0:
            isNotFraud += 1
        else:
            fraud += 1
print('Elements in List: {}\nElements fraudulent: {}\nElements not fraudulent: {}'.format(general_count, fraud, isNotFraud))

Elements in List: 1091
Elements fraudulent: 88
Elements not fraudulent: 1003


In [31]:
final_df = result_data[0]

In [32]:
for i in range(1, len(result_data)):
    final_df = final_df.append(result_data[i], ignore_index=True)
final_df.head()

,credit_card,transaction_id,amount_spent,belonging_to_cluster,transaction_is_fraudulent,transaction_time_delta,credit_card_holder,credit_card_expiration_date,crdeit_card_validation_code_two
0,1010,3378279,277.932007,2.0,0,9826142,555.0,121.0,226.0
1,1060,3113789,387.950012,2.0,0,2507926,512.0,150.0,226.0
2,1096,3409488,349.950012,2.0,0,10698273,275.0,150.0,224.0
3,1096,3502806,554.000000,2.0,0,13489746,275.0,150.0,224.0
4,1099,3573933,250.000000,2.0,0,15708121,399.0,150.0,185.0


Nun ist das DataFrame für das Training des KNN fertiggestellt. Im Anschluss werden nun die Labels aus dem <code>DataFrame</code> in ein separates 1-dimensionales <code>numpy</code> array geschrieben. Daran anschließened werden alle kategorischen Daten aus dem <code>DataFrame</code> entfernt, sodass nur dieses nur noch aus den Spalten <code>amount_spent</code> und <code>transaction_time_delta</code> besteht.

In [33]:
train_df = final_df[:654]
test_df = final_df[655:]

In [34]:
train_fraudulent_label_list = train_df['transaction_is_fraudulent'].values.tolist()
test_fraudulent_label_list = test_df['transaction_is_fraudulent'].values.tolist()
train_fraudulent_label_list = np.asarray(train_fraudulent_label_list)
test_fraudulent_label_list = np.asarray(test_fraudulent_label_list)
print(type(train_fraudulent_label_list))
print(type(test_fraudulent_label_list))

<type 'numpy.ndarray'>
<type 'numpy.ndarray'>


In [35]:
cols_to_drop = ['credit_card', 'transaction_id', 'belonging_to_cluster', 'transaction_is_fraudulent', 'credit_card_holder', 'credit_card_expiration_date', 'crdeit_card_validation_code_two']
modified_train_df = train_df.drop(columns=cols_to_drop)
modified_test_df = test_df.drop(columns=cols_to_drop)

Dieses kann nun normalisiert werden.

In [36]:
min_max_scaler = preprocessing.MinMaxScaler()

In [37]:
modified_train_df_as_numpy = modified_train_df.values
modified_test_df_as_numpy = modified_test_df.values
modified_train_df_as_numpy_normalized = min_max_scaler.fit_transform(modified_train_df_as_numpy)
modified_test_df_as_numpy_normalized = min_max_scaler.fit_transform(modified_test_df_as_numpy)
normalized_modified_train_df = pd.DataFrame(modified_train_df_as_numpy_normalized)
normalized_modified_test_df = pd.DataFrame(modified_test_df_as_numpy_normalized)

## KNN
Im weiteren Verlauf wird ein Feed Forward Neural Net mit nur einer Hidden Layer erstellt:
<ul>
    <li>Input Layer: 1 Node</li>
    <li>Hidden Layer: 2/4/8 Nodes, mit unterschiedlichen <a href="https://www.tensorflow.org/api_docs/python/tf/keras/activations">Aktivierungsfunktionen</a></li>
    <li>Output LayeR: 2 Nodes (jeweils eine Node für <i>fraud</i> bzw. <i>no fraud</i> )
</ul>
Dabei wird explorativ vorgegangen. Das bedeutet, dass die Hidden Layer in mehreren Iterationen mit verschiedenen Aktiviuerungsfunktionen geprüft wird. Außerdem wird jede Aktivierungsfunktion mit einer unterschiedlichen Anzahl von verbundenen Neuronen geprüft. 

In [499]:
model = keras.Sequential([
    keras.layers.Dense(1),
    keras.layers.Dense(8, activation='tanh'),
    keras.layers.Dense(2, activation='softmax')
])

In [500]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [501]:
model.fit(normalized_modified_train_df.values, train_fraudulent_label_list, epochs=4)

Epoch 1/4
654/654 [==============================] - 1s 895us/sample - loss: 0.6612 - acc: 0.8639
Epoch 2/4
654/654 [==============================] - 0s 60us/sample - loss: 0.6081 - acc: 0.9052
Epoch 3/4
654/654 [==============================] - 0s 59us/sample - loss: 0.5533 - acc: 0.9052
Epoch 4/4
654/654 [==============================] - 0s 59us/sample - loss: 0.4980 - acc: 0.9052


In [502]:
test_loss, test_acc = model.evaluate(normalized_modified_test_df.values, test_fraudulent_label_list, verbose=2)
print('\nTest accuracy:', test_acc)

436/436 - 1s - loss: 0.4474 - acc: 0.9404

Test accuracy: 0.940367


In [498]:
model = None